#Main file
This main file is based on the scripts that I had for minimizing the machines from Netlogo. The original was made to capture output from Behaviour Space, processs the machines and then print it to use in Stata. This is too cumbersome, so decided to implement and do everything in Python so I can centralise all the analysis and work on the next algorithms such as Joint Machines, frequencies and unused behavioural states in order to analyse properly how the transitions are happening.

#1 Receiving the Netlogo output
The first step is to load the files. Also, as the global variables of interest in order to associate it with the output name file.

In [1]:
import pandas as pd
import numpy as np
import copy
from __future__ import division
import minimization as minz #My script in same folder for minimization routines


#Choose here the Globals and Name used for the experiment to load.
#Make sure the files exist (i.e. from Netlogo simulations)
experiment_clue = "trial1"
n_states = 8
n_signals = 1 #only without signal for now
n_rounds = 20
N = 40
n_parents = 20

globals_list = (experiment_clue, n_states, n_signals, n_rounds, N, n_parents) #Save them as a list

#Experiment name based on the chosen experiment_clue and globals
chosen_experiment = "%s_states_%s_signal_%s_rounds_%s_N_%s_parents_%s.txt" % globals_list

#Both files have to use the same "chosen experiment" (to make sure come from the same simulation in Netlogo)
summary_file_name = "summary_" + chosen_experiment #Summary output
strategies_file_name = "strategies_" + chosen_experiment #Strategies output

#Path to Netlogo outputs
netlogo_folder = "/Users/luisalejandrolee/Dropbox/Thesis Phd/\
Coordination autos Chapter three/outputs_from_netlogo/" #Netlogo outputs in this folder

#Read files and save them as data
df_sum = pd.read_csv(netlogo_folder + summary_file_name)
df_strat = pd.read_csv(netlogo_folder + strategies_file_name)

#2 Minimise the automata
Use the functions to have a simple code here for minimising the auto and storing other relevant variables (as available states, etc)

In [2]:
#Other required globals
n_obs = 2 if n_signals == 0 else 4 #Define here (or change) the possible observations of the machines

canon_autos_list = [] #Create empty lists to store the processed autos below
min_autos_list = []


for i in df_strat.index: #For each row...
    netlogo_auto = df_strat.auto_long[i] #... for all netlogo_auto
    big_auto = minz.to_format_netlogo_auto(netlogo_auto) #Use function to convert the raw Netlogo auto in a list format
    init_state = big_auto[0] #Save initial state of the machine
    normal_auto = minz.new_empty_auto(n_obs, n_states) #Use function to create a new empty auto as a numpy array

    # Next block it to fill the new 'normal_auto' with the information from big_auto.
    # The objective is that normal_auto=big_auto but as an array (instead of a list)
    my_index = xrange(1, len(big_auto), n_obs + 1) # Each number in the index is where a state starts
    for i, j in enumerate(my_index):
        normal_auto['actions'][i] = big_auto[j]
        normal_auto['transitions'][i] = big_auto[j + 1:j + n_obs + 1]
        
    canon_auto = minz.convert_to_canonical(normal_auto, n_states, init_state, n_obs) #Use function for canonical form
    
    access_states = len(canon_auto) #n_states now is only the accesible states of the machine (before minimization)
    
    #Use function to get minimum behavioural equivalent auto
    #Passes "0" as 3rd argument because that's init_state now (always 0 for canonical auto)
    min_auto = minz.minimized_automaton(canon_auto, access_states, 0, n_obs)
    
    #Update autos lists
    canon_autos_list.append(canon_auto) #Store proccessed autos in the corresponding list
    min_autos_list.append(min_auto)
    
#Add the processed autos lists as columns to df_strat
df_strat["canon_autos"] = canon_autos_list #Add the lists with autos to the dataframe
df_strat["min_autos"] = min_autos_list

#3 Accesible states and minimum behavioural states

In [3]:
access_states = [len(x) for x in df_strat.canon_autos] #accesible states in the big machine
min_states = [len(x) for x in df_strat.min_autos] #accesible states in the minimized machine

df_strat["access_states"] = access_states
df_strat["min_states"] = min_states

#4 Joint machine (not minimized) and Unused States measure (WORKS ONLY WITHOUT SIGNAL)
Outputs the df_jms including the NOT minimized joint machines.

Outputs on df_strat the 'used_states' variable (containing list of used and unused states of minimized individual 
machines

In [4]:
#Prepare dataframe to keep track of unused states
df_strat["used_states"] = 0 # Will contain a list with states of each min_auto
used_states_list = []
for i, auto in enumerate(df_strat.min_autos): #all minimized autos
    a = [0 for ix in xrange(df_strat.min_states[i])] #List the size of minimised machine's states
    used_states_list.append(a)
df_strat["used_states"] = used_states_list #Add to dataframe

# Lists to keep track of joint machines
gen_list = []
jm_list = []


In [5]:
auto0 = df_strat.min_autos[1758] #Autos to pass to create_joint_machine function
auto1 = df_strat.min_autos[1759]

In [76]:
def create_joint_machine_with_signal(auto0, auto1):
    
    temp = [None, None]
    size = len(auto0) * len(auto1) #one metastate for each combination of states of component autos
    states = [temp[:] for i in xrange(size)]#all pairs of states of component autos. The position in list 
                                            #uniquely identifies each possible combination
    jm=[[None,None] for i in xrange(size)]#joint machine (each state is action(e.g. AA or BA) and 
                                            #transitions (first for signal "Heads", then for "Tails"))
    i=0
    for ix0, st0 in enumerate(auto0):
        for ix1, st1 in enumerate(auto1):#all combinations of states of individual autos
            states[i]=[ix0,ix1]
            jm[i][0]=[st0['actions']+st1['actions']]
            i+=1
    
    jm[0][1]=[0,0] #first metastate is initial states of individual autos
    
    for ix,ms in enumerate(jm): #all meta states of the joint machine
        auto0_state=states[1][0]#state and action of each auto for the current metastate
        auto1_state=states[1][1]
        auto0_action=ms[0][0][0]
        auto1_action=ms[0][0][1]
        
        #Get next state (transition) for each auto, based on current state, rival's action and signal
        auto0_next_st_heads=update_state_with_signal(auto0, auto0_state, auto1_action, 'H')#auto,state,rival_action,signal
        auto1_next_st_heads=update_state_with_signal(auto1, auto1_state, auto0_action, 'H')
        auto0_next_st_tails=update_state_with_signal(auto0, auto0_state, auto1_action, 'T')
        auto1_next_st_tails=update_state_with_signal(auto1, auto1_state, auto0_action, 'T')
        
        indiv_trans_heads=[auto0_next_st_heads,auto1_next_st_heads]#if Heads, each autos transitions
        indiv_trans_tails=[auto0_next_st_tails,auto1_next_st_tails]
        
        #the key is that each possible combination of states is uniquely associated with one position
        #in the "states" list. So given the two individual states, "states.index" gives the position of the state
        #which is interpreted as the transition for the metamachine
        meta_trans_heads=states.index(indiv_trans_heads)
        meta_trans_tails=states.index(indiv_trans_tails)
        
        meta_trans=[meta_trans_heads,meta_trans_tails]
        
        jm[ix+1][1]=meta_trans[:]#copy metatransition to next metastate

        if ix==size-2:#in last state, no need to update any more
            break
      
    return jm

jm = create_joint_machine_with_signal(auto0, auto1)
jm
#track unused states:
#make jm an array
#make it canonical
#minimize it
#organize functions in minz
#do the same for without signal

[[['AA'], [0, 0]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AB'], [2, 10]],
 [['AB'], [2, 26]],
 [['AB'], [2, 26]],
 [['AA'], [2, 26]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AB'], [2, 10]],
 [['AB'], [2, 26]],
 [['AB'], [2, 26]],
 [['AA'], [2, 26]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AB'], [2, 10]],
 [['AB'], [2, 26]],
 [['AB'], [2, 26]],
 [['BA'], [2, 26]],
 [['BA'], [2, 13]],
 [['BA'], [2, 13]],
 [['BA'], [2, 13]],
 [['BA'], [2, 13]],
 [['BB'], [2, 13]],
 [['BB'], [2, 29]],
 [['BB'], [2, 29]],
 [['BA'], [2, 29]],
 [['BA'], [2, 13]],
 [['BA'], [2, 13]],
 [['BA'], [2, 13]],
 [['BA'], [2, 13]],
 [['BB'], [2, 13]],
 [['BB'], [2, 29]],
 [['BB'], [2, 29]],
 [['AA'], [2, 29]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AA'], [2, 10]],
 [['AB'], [2, 10]],
 [['AB'], [2, 26]],
 [['AB'], [2, 26]],
 [['AA'], [2, 26]],
 [['AA'], [2, 10]],
 

In [73]:
#returns the next state for an auto, given current_state, rival's action and signal
def update_state_with_signal(auto, current_state, rival_action, signal):
    if rival_action == "A" and signal=='H':
        trans = 0
    elif rival_action == "B" and signal=='H':
        trans = 1
    elif rival_action == "A" and signal=='T':
        trans = 2
    elif rival_action == "B" and signal=='T':
        trans = 3
    new_state = auto[current_state][1][trans]

    return new_state



In [ ]:

#Create the joint machines (not minimized) and track unused states
#Main code of this part

for gen in df_sum.generation: #each generation
    #======HERE FOR PRINTING TIME============ 
    print "generation = ", gen #Useful for debugging (or keeping track of simulation time)
    #Next line (the "for" block) is tricky:
    #It creates a dataframe containing only the autos with population = column and current generation.
    #Uses "iterrows()" to iterate over the index of the dataframe (df_strat), keeping it on i0, which is
    #needed to acces later the particular auto that was used for the joint machine (accesed by row0.min_autos)
    #This is used (instead of a simple enumerate) to access that row later when updating the used states 
    #Same logic for the second "for" block, but for population row
    
    for i0, row0 in df_strat[(df_strat.population == "column") & (df_strat.generation == gen)].iterrows():
        for i1, row1 in df_strat[(df_strat.population == "row") & (df_strat.generation == gen)].iterrows():
            
            #print "generation = ", gen, "i0 = ", i0, "i1 = ", i1
            auto0 = row0.min_autos #Autos to pass to create_joint_machine function
            auto1 = row1.min_autos
            
            #Joint machine only work WITHOUT signal so far (change "create_joint_machine" and "update_state" functions)
            jm = minz.create_joint_machine_no_signal(auto0, auto1) #Function to create the joint machine
            
            gen_list.append(gen) #Keeps track of generation
            jm_list.append(jm)   #Keeps track of joint machines
            
            #Update used_states:
            #This part takes the information from the just create joint machine jm, which contains the states that
            #are visited by the two autos that created it. For each auto, then goes trough each state. The
            #list "used_states" contains a list of zeros with the number of states of the auto, each position
            #representing each state of the auto. So if state 1 of the machine is used in the joint machine, then
            #position 1 will be changed from zero to one, to indicate that the state is used.
            
            for st in jm["states"]: #for metastates in jointmachine
                s0 = st[0] #state of auto0
                df_strat["used_states"][i0][s0] = 1 # 1 for states visited. Unvisited remain 0
            
            for st in jm["states"]:
                s1 = st[1] #state of auto1
                df_strat["used_states"][i1][s1] = 1


#Store the joint machines and generation (a new dataframe)
df_jms = pd.DataFrame(columns = ("generation", "jm")) #Store joint machines with associated generation
df_jms.generation = gen_list
df_jms.jm = jm_list

#5 Minimize the joint machines (without signal)
Adds the 'min_jm' column to df_jms (tuples with the minimized joint machines)

In [5]:
min_jm_list = [] #to save the minimized joint machines, and add later to the dataframe (df_jms)

for jm in df_jms.jm: #all joint machines
#for jm in [df_jms.jm[24000], df_jms.jm[24001]]:
    min_jm = minz.minimize_joint_machine_no_signal(jm) #minimize them (have actions and cyclestart)
    #print min_jm
    
    #Next lines convert the min_jms, which is a dict, into a tuple
    #Basically converts a list of lists ("actions" in the min_jm)into a tuple of tuples
    #This is so that it can be used as a key to use groupby (since tuple is inmutable)
    tup_actions = tuple(tuple(pair_actions) for pair_actions in min_jm["actions"]) #convert actions to tuples
    min_jm = (tup_actions, min_jm["cyclestart"]) #add the cyclestart to final min_jm tuple
    
    min_jm_list.append(min_jm) #save the minimized machine to a list
    #print min_jm
    
# Save the minimized joint machines
df_jms["min_jms"] = None #new empty column in dataframe
df_jms.min_jms = min_jm_list #add the minimized joint machines to the dataframe



#6 Frequencies of minimized joint machines
Outputs dataframe "freqjm" with frequencies of joint machines
(Does it by transforming df_jms)

In [6]:
#Use Groupby and organize the data set for frequencies

g1 = df_jms.copy() #use intermediate copies to avoid potential bugs later. Not sure if actually needed...
g1 = g1.groupby([g1["generation"], g1["min_jms"]]) #split by groups
g1 = g1.count() #organize as frequency of joint machine per generation

interactions = N * N #number of joint machines per generation
freq_perc_list = [(x*100)/interactions for x in g1.jm] #list with frequency percentage of jm per generation
g1['freq_perc'] = freq_perc_list #add frequency to the dataframe

#jm_freq_threshold = 0 #Change to higher for easier visualization
#g1 = g1[g1.freq_perc > jm_freq_threshold] #keep machines with frequency higher than threshold


#Organise the dataframe

freqjm = g1.copy() #just in case...
freqjm = freqjm.rename(columns = {'jm':'freq'}) #rename column
freqjm = freqjm.reset_index() #reset_index converts the multiindex into normal columns (to use generation for 'sort')
freqjm = freqjm.sort(['generation', 'freq_perc'], ascending=[True, False]) #sort
jmstring = [minz.min_jm_no_signal_to_string(x) for x in freqjm.min_jms] #use function to convert to string
freqjm['jmstring'] = jmstring #add to dataframe
#freqjm

#7 Unused behaviour and slack in construction measures

The measures are first captured for each machine in the dataframe df_strat.
However, they are then averaged across generation so that the averages can be copied to the summary dataframe.
So the measures in the latter are averages

In [7]:
#Unused states: number of states not visited in the minimized machine
unused_states = [len(x) - x.count(1) for x in df_strat.used_states] #unused states in min_autos

#Unvisited: potential for novel behavior given change in the  input stream. Is unused states divided by min_states 
unvisited_measure = [(len(x)-x.count(1))/len(x) for x in df_strat.used_states]

#Behaviour_slack: slack in the potential behavior of the machine
#the more states you use, the more sophisticated you can become behaviorally.
behaviour_slack = [len(x)/n_states for x in df_strat.min_autos] #min_lenght/total states.

#construction_slack: slack in the construction of the complete machine
construction_slack = [x/n_states for x in df_strat.access_states]#accesible/total

df_strat['unused_states'] = unused_states
df_strat['unvisited_measure'] = unvisited_measure
df_strat['behaviour_slack'] = behaviour_slack
df_strat['construction_slack'] = construction_slack

#df_strat = df_strat.drop('used_states', 1)

In [8]:
#Take the average per generation of unused states, unvisited measure, and slack measures

strats = df_strat.copy() #just in case
strats = strats.groupby(strats.generation).mean() #take the mean of all the variables (by generation)
strats = strats.drop(['ID','score',],1) #not needed
strats = strats.reset_index()

In [9]:
#Organise summary dataframe, and include the measures calculated above.

summary = df_sum.copy() #just in case
#delete columns that won't use
to_delete = ['row_heads_A', 'row_heads_B', 'row_tails_A', 'row_tails_B', 'col_heads_A', 'col_heads_B',\
'col_tails_A','col_tails_B','times_heads','times_tails']
summary = summary.drop(to_delete, axis=1)

summary = pd.merge(summary, strats, on='generation') #merge datasets


#8 Regime identification
A regime is defined as the most frequent minimized joint machine per generation, if that machine has a minimum threshold of frequency percentage. Otherwise, the regime is classified as "other"

In [10]:
#Function to find the highest frequency percentage top machine
def find_top_jm (df, n=1, column='freq_perc'):
    return df.sort_index(by=column)[-n:]

#Apply the function to get the highest frequency joint machine per generation
topjm = freqjm.groupby('generation').apply(find_top_jm)

#Define regime as the top joint machine in a generation if its frequency is above the defined "regime_threshold"
#percentage. If none is above it, the regime is in "other"
regime_threshold = 50 
regime = [jm if int(topjm.freq_perc[i]) > regime_threshold else 'not_threshold' for i,jm in enumerate(topjm.jmstring)]

#Add regime to summary dataframe
summary['regime'] = regime

#9 Epoch characterization
An epoch is defined as having the top joint machine on a generation (i.e. regime) to be the same over a window of 
past generations (e.g. was this period the same regime as in the past ones?). In that window of past regimes, some tolerance is permitted (i.e. some of them can be different, allowing for some errors).

The algorithm considers an epoch to have started when a regime appears a minimum number of times in the window of past regimes.

In [11]:
epochs = pd.DataFrame(columns=['epoch','duration','start','end']) #to be filled with all epoch information
#Parameters that can be changed
epoch_window = 3 #lagged regimes to be considered
epoch_tolerance = 1 #number of misses in the window before breaking an epoch

switch = False #is there an epoch already started?
length = 0 #of current epoch
current_epoch = {'epoch': '', 'duration': 0, 'start': 0, 'end': 0}#will store info of each epoch

for t,ep in enumerate(summary.regime):#all generation (or regimes)
    if t >= epoch_window: #intial observations not considered due to lagged window
        lags = [summary.regime[tt] for tt in xrange(t-epoch_window, t)] #window of past generations
        
        
        #BEGIN:An epoch begins. This is when switch is False (no epoch has started) and epoch criteria is fulfilled
        if lags.count(ep) >= (epoch_window-epoch_tolerance) and switch == False: #an epoch begins
            switch = True
            current_epoch['epoch'] = ep
            length += 1
            current_epoch['duration'] = length
            current_epoch['start'] = t
            #print 'begins ', current_epoch
        
        #CONTINUE:An epoch already is going, and criteria is met
        elif lags.count(ep) >= (epoch_window-epoch_tolerance) and switch == True: #an epoch continues
            length += 1
            current_epoch['duration'] = length           
            #print 'continues ', current_epoch
        
        
        #END:An epoch is going, but criteria is not met. This could also mean there is an exception.
        #An exception is a regime that shows up few times in a row (formally,less than the epoch_tolerance).
        #The objective is that an exception doesn't end up and epoch. Initially, exceptions end them. However,
        #This is dealt with later after the whole dataframe (epochs) is constructed.
        elif lags.count(ep) < (epoch_window-epoch_tolerance) and switch == True:
            current_epoch['end'] = t
            index = len(epochs.index)
            epochs.loc[index+1]=current_epoch
            
            switch = False #reset variables
            length = 0
            current_epoch = {'epoch': '', 'duration': 0, 'start': 0, 'end': 0}
            #print 'end ', current_epoch

        if t == len(summary.index)-1 and switch == True: #last generation that is not the end of an epoch
            current_epoch['end'] = None
            index = len(epochs.index)
            epochs.loc[index+1]=current_epoch
            
        #print 'lags', lags
    #print 'regime = ', ep,'\n'
    
#Once 'epochs' is finished, handle the exceptions:
#Take each row, and compare starting generation of an epoch with ending generation of the previous one.
#If they are close enough (dictated by epoch_tolerance), join both epochs (rows) as a single one

epochs_debug = copy.deepcopy(epochs)#capture for debugging before altering

for i in xrange(2, len(epochs.index)+1): #no zero position considered due to epochs starting at 1 (append coding)
    regime_t = copy.deepcopy(epochs.epoch[i]) #variables from the dataframe to manipulate
    regime_lag = copy.deepcopy(epochs.epoch[i-1])
    end_t = copy.deepcopy(epochs.end[i])
    start_t = copy.deepcopy(epochs.start[i])
    end_lag = copy.deepcopy(epochs.end[i-1])
    start_lag = copy.deepcopy(epochs.start[i-1])
    
    if regime_t == regime_lag and start_t-end_lag <= epoch_tolerance:#if considered as same epoch
        epochs = epochs.drop(i-1)
        epochs.loc[i, 'start'] = start_lag
        epochs.loc[i, 'duration'] = end_t-start_lag

#10 Export main dataframes
Export the three main data frames, so I can work with graphs and statistics from a different file. This makes the scripts a bit more modular, and also I just have to run the minimization procedures (this file) only once per experiment.

In [12]:
#Path to python outputs
python_folder = "/Users/luisalejandrolee/Dropbox/Thesis Phd/\
Coordination autos Chapter three/outputs_from_python/" #Netlogo outputs in this folder

#summary: contains main variables. Averages per generation
summary.to_csv(python_folder+'summary_'+chosen_experiment)

#freqjm: frequency of each joint machine per generation.
freqjm.to_csv(python_folder+'jm_'+chosen_experiment)

#epochs: each row has the regime (top joint machine if above threshold), start and end generation, and duration
epochs.to_csv((python_folder+'epochs_'+chosen_experiment))

#STOP! no need to run the script further
###Just for visualization
freqjm: frequency of all joint machines per generation (for transition analysis)

summary: main variables (averages per generation)

epochs: classification of epochs, with duration, starting and ending periods.

In [12]:
#For visualization of joint machines with different frequency thresholds

jm_freq_threshold = 0 #Change to higher for easier visualization (percentage)
freqjm[freqjm.freq_perc > jm_freq_threshold] #keep machines with frequency higher than threshold



,generation,min_jms,freq,freq_perc,jmstring
728,0,"(((B, B),), 0)",72,4.5000,>> BB <<
390,0,"(((B, A),), 0)",56,3.5000,>> BA <<
0,0,"(((A, A),), 0)",45,2.8125,>> AA <<
391,0,"(((B, A), (A, A)), 1)",35,2.1875,BA >> AA <<
221,0,"(((A, B),), 0)",25,1.5625,>> AB <<
690,0,"(((B, A), (B, B), (B, A)), 0)",21,1.3125,>> BA BB BA <<
652,0,"(((B, A), (B, B)), 1)",18,1.1250,BA >> BB <<
183,0,"(((A, A), (B, B)), 1)",16,1.0000,AA >> BB <<
127,0,"(((A, A), (B, A)), 1)",15,0.9375,AA >> BA <<
128,0,"(((A, A), (B, A), (A, A)), 2)",14,0.8750,AA BA >> AA <<


In [13]:
summary

,generation,av_score_row,av_score_col,miscoordination_perc,coordination_B_perc,coordination_A_perc,ce,ce_individual,access_states,min_states,unused_states,unvisited_measure,behaviour_slack,construction_slack,regime
0,0,1.310125,1.304875,0.477000,0.264125,0.258875,0,0,6.3250,5.6875,0.0000,0.000000,0.710938,0.790625,not_threshold
1,1,1.528719,1.393469,0.415563,0.359844,0.224594,0,0,6.4000,5.9125,0.0000,0.000000,0.739062,0.800000,not_threshold
2,2,2.139281,1.800875,0.211969,0.563219,0.224812,0,0,6.7000,6.4000,0.0875,0.012351,0.800000,0.837500,not_threshold
3,3,2.841937,1.905406,0.050531,0.943000,0.006469,0,0,6.5250,6.2875,0.2875,0.035937,0.785937,0.815625,not_threshold
4,4,2.632563,1.759781,0.121531,0.875625,0.002844,0,0,6.7750,6.4375,0.2875,0.037723,0.804688,0.846875,not_threshold
5,5,2.881375,1.928625,0.038000,0.957375,0.004625,0,0,7.1375,6.7375,0.5375,0.071875,0.842187,0.892188,not_threshold
6,6,2.674250,1.834500,0.098250,0.870750,0.031000,0,0,7.3625,7.1750,1.6625,0.232738,0.896875,0.920312,not_threshold
7,7,2.863906,1.914219,0.044375,0.952656,0.002969,0,0,7.3125,6.9625,0.4625,0.062500,0.870313,0.914062,not_threshold
8,8,2.736656,1.847562,0.083156,0.902969,0.013875,0,0,7.3625,7.0375,1.8625,0.264211,0.879687,0.920312,not_threshold
9,9,2.996437,1.997625,0.001187,0.998812,0.000000,0,0,7.3250,6.9500,3.3250,0.467798,0.868750,0.915625,BB AB BA BB AB >> BB <<


In [34]:
epochs

,epoch,duration,start,end
2,not_threshold,10,3,13
3,>> BB <<,16,15,None
